In [1]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# CIFAR-10 содержит 60К изображений 32*32*3 канала цвета
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

BATCH_SIZE = 128
NB_EPOCH = 25
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
# Пребобразуем к категориальному виду
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

In [5]:
# Преобразуем к формату с плавующей точкой и нормируем к диапазоу (0,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [6]:
# Описываем нейросеть
model = Sequential()
model.add(Conv2D(32, (7, 7),padding='same',
    input_shape=(IMG_ROWS,IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4736      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
activation_2 (Activation)    (None, 512)              

In [7]:
# Обучение модели
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
    metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE,
    epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT,
    verbose=VERBOSE)

score = model.evalaute(X_test, Y_test, 
    batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print("Test accuracy", score[1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 98s 2ms/step - loss: 1.7577 - accuracy: 0.3796 - val_loss: 1.5047 - val_accuracy: 0.4682
Epoch 2/25
40000/40000 [==============================] - 86s 2ms/step - loss: 1.4040 - accuracy: 0.5043 - val_loss: 1.2681 - val_accuracy: 0.5618
Epoch 3/25
40000/40000 [==============================] - 90s 2ms/step - loss: 1.2474 - accuracy: 0.5641 - val_loss: 1.2069 - val_accuracy: 0.5817
Epoch 4/25
40000/40000 [==============================] - 90s 2ms/step - loss: 1.1500 - accuracy: 0.5967 - val_loss: 1.1313 - val_accuracy: 0.6071
Epoch 5/25
40000/40000 [==============================] - 93s 2ms/step - loss: 1.0650 - accuracy: 0.6273 - val_loss: 1.2014 - val_accuracy: 0.5931
Epoch 6/25
40000/40000 [==============================] - 90s 2ms/step - loss: 1.0054 - accuracy: 0.6520 - val_loss: 1.1114 - val_accuracy: 0.6234
Epoch 7/25
40000/40000 [==============================] - 91s 2ms/st

AttributeError: 'Sequential' object has no attribute 'evalaute'

In [ ]:
model_json = model.to_json()
open('cifar10_arch.json','w').write(model_json)
model,save_weights('cifar10_weights.h5', overwrite = True)